## 1. Установка необходимых библиотек, включая MediaPipe и OpenCV:

In [ ]:
!pip install mediapipe opencv-python-headless

In [ ]:
import gdown, os, shutil                                                        # Системные инструменты
from tqdm.auto import tqdm                                                      # Модуль для создания прогресс-бара при выполнении циклов
import numpy as np
import mediapipe as mp                                                          # Библиотека MediaPipe
import cv2                                                                      # Библиотека обработки изображений

Из-за проблем с распаковкой архивов с файлами в кирилице качаем не архив, а папку с файлами

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#shutil.copytree('/content/drive/MyDrive/ИИ', '/content/ИИ')
shutil.copytree('/content/drive/MyDrive/ИИ/МАВАШИ ГЕРИ  (КРУГОВОЙ УДАР ЗАДНЕЙ НОГОЙ)/ФРОНТАЛЬНО', '/content/ИИ/МАВАШИ ГЕРИ  (КРУГОВОЙ УДАР ЗАДНЕЙ НОГОЙ)/ФРОНТАЛЬНО')

## 3. Получаем список всех видеофайлов в папке "ИИ" и ее подпапках

In [ ]:
video_files = []
for root, dirs, files in sorted(os.walk('/content/ИИ')):
    for file in files:
        video_files.append(os.path.join(root, file))
video_files

## 4. Функция обрабатки видеофайла

In [ ]:
mp_pose = mp.solutions.pose                                                     # Создаем объекты для обнаружения скелетных точек на изображении
mp_mask = mp.solutions.holistic                                                 # Создаем объекты для наложение маски на изображении
mp_drawing = mp.solutions.drawing_utils                                         # Создаем объекты для рисование ключевых точек на кадре

def process_video1(video_file, size=(1280, 720)):
    folder = video_file.split(sep='/')[-4:]                                     # Разделяем путь к файлу на части: имя файла и имена папок
    filename_no_ext = os.path.splitext(folder[3])[0]                            # Получаем имя файла без расширения, чтобы использовать его в качестве идентификатора
    video_file_out = f'/content/{folder[0]}/{folder[1]}/{folder[2]}/{filename_no_ext}.point.mp4'
    #video_file_out = f'{filename_no_ext}.mp4'
    
    pose = mp_pose.Pose(model_complexity=2,                                     # задаем параметры для 'pose'
                        min_detection_confidence=0.5,                             
                        min_tracking_confidence=0.5)
    mask = mp_mask.Holistic(  model_complexity=2,                               # задаем параметры для 'mask'
                        enable_segmentation=True)
    
    cap = cv2.VideoCapture(video_file)                                          # Инициализация чтения видео файла
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))                        # Получаем количество кадров в видео файле
    fps = int(cap.get(cv2.CAP_PROP_FPS))                                        # Получаем частоту кадров

    record = cv2.VideoWriter(                                                   # Инициализация записи видео файла
                          video_file_out,                                       # Путь записи видео файла
                          cv2.VideoWriter_fourcc('M','P','4','V'),              # 4-значный код кодека для сжатия кадров
                          fps,                                                  # количество кадров в секунду
                          size,                                                 # размер кадров 
                          True)
            
    for i in tqdm(range(frame_count)):                                          # Пробегаем по каждому кадру в видео файле
        ret, image = cap.read()
        if not ret:                                                             # Если кадр не считался, то прекращаем цикл
              break

        image = cv2.resize(image, (size))                                       # Приводим размер изображения к размеру видео                 
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)                          # Переводим картинку в RGB

        results1 = mask.process(image)                                          # С помощью метода process выполняется сегментация на переднем и заднем планах 
        #condition = np.stack((results1.segmentation_mask,) * 3, axis=-1) > 0.5 # Булева маска сегментации на 2 канала с порогом 0,5
        if results1.segmentation_mask is not None:
            condition = np.stack((results1.segmentation_mask,) * 3, axis=-1) > 0.5
        else:
            print('В results1.segmentation_mask ошибка')
            #results1.segmentation_mask = np.nan_to_num(results1.segmentation_mask, nan=0.0)
            results1.segmentation_mask = np.zeros_like(results1.segmentation_mask)
            condition = np.stack((results1.segmentation_mask,) * 3, axis=-1) > 0.5

        bg_image = np.zeros(image.shape, dtype=np.uint8)                        # Изображение заднего плана, заполненное нулями
        image = np.where(condition, image, bg_image)                            # Выходное изображение содержит передний план, если значение в маске сегментации равно 1, и задний план, если значение равно 0

        results2 = pose.process(image)                                          # Обрабатываем изображение для обнаружения скелетных точек
        mp_drawing.draw_landmarks(image,                                        # Рисуем ключевые точки на кадре
                                  results2.pose_landmarks, 
                                  mp_pose.POSE_CONNECTIONS)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)                          # Конверуем кадр обратно в BGR
        record.write(image)                                                     # Записываем текущий кадр         
    cap.release()                                                               # Закрываем захват видео
    record.release()                                                            # Закрываем запись
    pose.close()
    mask.close() 

    os.remove(video_file)                                                       # удаляем исходный видео-файл
    os.rename(video_file_out, video_file)                                       # переименовываем/перемещаем созданный файл на место удаленного 
                                            

## 5. Обрабатываем видеофайлы из списка и записываем видео с скелетными точками

In [ ]:
size = (1280, 720)                                                              # Размер сохраняемого видео

for video_file in tqdm(video_files):
    #print(video_file)
    process_video1(video_file, size)